# Salary Analysis Project

This notebook performs a comprehensive analysis of salary data including EDA, statistical testing, and linear regression modeling.

## Table of Contents
1. Data Loading and Initial Exploration
2. Exploratory Data Analysis (EDA)
3. Data Preprocessing
4. Hypothesis Testing
5. Model Development
6. Model Evaluation
7. Model Interpretation

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
import shap

# Set style for visualizations
plt.style.use('seaborn')
sns.set_palette('husl')

# Configure pandas display options
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

## 1. Data Loading and Initial Exploration

In [ ]:
# Load the dataset
df = pd.read_csv('salaries.csv')

# Display basic information about the dataset
print("Dataset Info:")
print("=============\n")
print(df.info())

print("\nFirst few rows:")
print("=============\n")
print(df.head())

# Separate numerical and categorical columns
numerical_features = ['work_year', 'salary', 'salary_in_usd', 'remote_ratio']
categorical_features = ['experience_level', 'employment_type', 'job_title', 'salary_currency',
                       'employee_residence', 'company_location', 'company_size']

## 2. Exploratory Data Analysis (EDA)

In [ ]:
# Summary statistics for numerical variables
print("Summary Statistics for Numerical Variables:")
print(df[numerical_features].describe())

# Distribution of salary_in_usd
plt.figure(figsize=(12, 6))
sns.histplot(data=df, x='salary_in_usd', bins=50)
plt.title('Distribution of Salaries (USD)')
plt.show()

# Box plot of salary by experience level
plt.figure(figsize=(12, 6))
sns.boxplot(data=df, x='experience_level', y='salary_in_usd')
plt.title('Salary Distribution by Experience Level')
plt.xticks(rotation=45)
plt.show()

# Correlation analysis
correlation_matrix = df[numerical_features].corr()
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')
plt.title('Correlation Matrix of Numerical Features')
plt.show()

# Categorical variable analysis
for cat_feature in categorical_features:
    plt.figure(figsize=(12, 6))
    df[cat_feature].value_counts().plot(kind='bar')
    plt.title(f'Distribution of {cat_feature}')
    plt.xticks(rotation=45)
    plt.show()

## 3. Data Preprocessing

In [ ]:
# Function to detect outliers using IQR method
def detect_outliers(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]

# Remove outliers from salary_in_usd
df_cleaned = detect_outliers(df, 'salary_in_usd')

# Prepare features for modeling
X = df_cleaned.drop(['salary_in_usd', 'salary', 'salary_currency'], axis=1)
y = df_cleaned['salary_in_usd']

# Create preprocessing pipeline
numeric_features = ['work_year', 'remote_ratio']
categorical_features = ['experience_level', 'employment_type', 'job_title',
                       'employee_residence', 'company_location', 'company_size']

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(drop='first', sparse=False), categorical_features)
    ])

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## 4. Hypothesis Testing

In [ ]:
# ANOVA test for salary differences across experience levels
experience_groups = [group for _, group in df_cleaned.groupby('experience_level')['salary_in_usd']]
f_statistic, p_value = stats.f_oneway(*experience_groups)
print("ANOVA Test Results:")
print(f"F-statistic: {f_statistic}")
print(f"p-value: {p_value}")

# Test for normality of salary distribution
_, normality_p_value = stats.normaltest(df_cleaned['salary_in_usd'])
print("\nNormality Test Results:")
print(f"p-value: {normality_p_value}")

## 5. Model Development

In [ ]:
# Simple Linear Regression
simple_model = LinearRegression()
X_simple = df_cleaned[['work_year']].values.reshape(-1, 1)
simple_model.fit(X_simple, y)

# Multiple Linear Regression
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])

# Fit the pipeline
pipeline.fit(X_train, y_train)

# Cross-validation
cv_scores = cross_val_score(pipeline, X_train, y_train, cv=5, scoring='r2')
print("Cross-validation scores:", cv_scores)
print("Mean CV Score:", cv_scores.mean())

## 6. Model Evaluation

In [ ]:
# Make predictions
y_pred = pipeline.predict(X_test)

# Calculate performance metrics
r2 = r2_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)

print("Model Performance Metrics:")
print(f"R² Score: {r2:.4f}")
print(f"MSE: {mse:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"MAE: {mae:.4f}")

# Residual analysis
residuals = y_test - y_pred
plt.figure(figsize=(10, 6))
sns.scatterplot(x=y_pred, y=residuals)
plt.axhline(y=0, color='r', linestyle='--')
plt.xlabel('Predicted Values')
plt.ylabel('Residuals')
plt.title('Residual Plot')
plt.show()

# Q-Q plot for residuals
plt.figure(figsize=(10, 6))
stats.probplot(residuals, dist="norm", plot=plt)
plt.title('Q-Q Plot of Residuals')
plt.show()

## 7. Model Interpretation

In [ ]:
# Get feature names after preprocessing
feature_names = (numeric_features +
                [f"{feature}_{val}" for feature, vals in
                 zip(categorical_features,
                     preprocessor.named_transformers_['cat'].categories_)
                 for val in vals[1:]])

# Get coefficients
coefficients = pd.DataFrame(
    pipeline.named_steps['regressor'].coef_,
    index=feature_names,
    columns=['Coefficient']
).sort_values('Coefficient', ascending=False)

# Plot feature importance
plt.figure(figsize=(12, 8))
sns.barplot(x=coefficients['Coefficient'], y=coefficients.index)
plt.title('Feature Importance (Coefficient Values)')
plt.show()

# SHAP values for feature importance
X_processed = pipeline.named_steps['preprocessor'].transform(X_test)
explainer = shap.LinearExplainer(pipeline.named_steps['regressor'], X_processed)
shap_values = explainer.shap_values(X_processed)

plt.figure(figsize=(12, 8))
shap.summary_plot(shap_values, X_processed, feature_names=feature_names)